### 1) Analyse brivement: 
* La base DVF enregistre les données des transactions immobilières de tous les communes en frances métropoles (hors Alsace et Moselle) ainsi que les département d'Outre-Mer (hors Mayotte). 
* À partir de ce jeux de données, nous pourons faire des analyses pour la valeur foncières en france selon différentes critères (typologie de biens, la nature de mutation,créer des outils cartographique interactive avec la valeur foncière par chaque communes et par le département (par exemple, comme les outils cartographiques que MeilleursAgents a utilisé, nous pourrons faire la carte de prix par m² des bureaux/commerces/entrepôts par communes en France). 
* Nous pourrons savoir aussi à partir de ce jeu de données l'évoution selon le temps ( entre 2014 et 2020 ) pour chaque type de biens.
* Le point faible de ce jeu, c'est qu'il n'y a pas mal des informations manquantes (comme les caractéristiques du bien, le nombre d'étage, ascenseur, année de construction ...) qui nous permet de faire une estimation la valeur du bien en utilisant la méthode hédonique et en prendre en compte l'impacte de la localisation spatial du bien.
* De plus,le jeu de données contient aussi beaucoup des données manquants.

### 2) Préparation des données:

Pour une étude quantitatives, la partie préparation des données joue un rôle très important. Plus on travail bien sur cette partie, plus on a les chances pour avoir les résultats pertinences dans les étapes analyses et modélisation, prédiction les données! Pour cette étape, nous avons réalisé les techniques comme:
               - Créer les dataset brutes et vérifier les données dans ce dataset.
               - Traitement des données manquantes et remplacer les données manquantes par méthode l'imputation.
               - Changement les types de certaines variables et choisir les variables importants pour analyser!
               - L'analyse exploratoire des données pour voir la rélation entre la valeur foncière et d'autres variables dans le jeu de données. Pour le traitement des outliers (données abérrantes) nous ne pouvons pas faire en raison des biens immobiliers dans ce jeu de données sont hétérogènes et de plus, pour la partie modélisation et la classification des données nous travaillons sur une autre jeu de données. C'est pour cette raison, je ne mets pas la partie traitement des outliers ici. 
             

In [ ]:
#Importer les packages python
import geopandas as gpd
import matplotlib
import pandas as pd
import numpy as np
import descartes
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
pd.options.mode.chained_assignment = None

In [ ]:
#Définir la chemin pour le jeu de données
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Ici, nous travaillons sur les transactions en 2020 comme une exemple
df_full20 = pd.read_csv('/kaggle/input/france-geolocated-dvf-land-values/2020-full.csv')

In [ ]:
#La base brute pour l'année 2020
df_full20.head(5)

In [ ]:
#Vérifier le taux des données manquantes dans la base brutes
df_full20.isnull().sum()/len(df_full20)*100

* Ici, nous constatons que la base brute contient beaucoup de variables qui ont un taux de plus de 90% des données manquantes. Donc, pour ces variables, nous avons exclué dans notre jeu de données. 
* Pour les autres variables qui ont un taux de moins de 40%, nous utilisons la méthode de l'imputation de données pour remplacer les données manquantes ( par exemple, on peut remplacer les données manquantes dans la valeur foncière par la valeur moyenne ou par le médian afin de ne pas changer la distribution de cette variables!)


In [ ]:
df_final = df_full20.drop(['numero_disposition',
                          'ancien_code_commune',
                          'adresse_suffixe',
                           'ancien_nom_commune',
                           'ancien_id_parcelle',
                           'numero_volume',
                           'lot1_numero',
                           'lot1_surface_carrez',
                           'lot2_numero',
                           'lot2_surface_carrez',
                           'lot3_numero',
                           'lot3_surface_carrez',
                           'lot4_numero',
                           'lot4_surface_carrez',
                           'lot5_numero',
                           'lot5_surface_carrez',
                           'code_nature_culture',
                           'nature_culture',
                           'code_nature_culture_speciale',
                           'nature_culture_speciale'],axis= 1)

In [ ]:
#Changer le types de variables
df_final['code_postal'] = df_final['code_postal'].astype('object')
df_final['code_type_local'] = df_final['code_type_local'].astype('object')
df_final['type_local'] = df_final['type_local'].astype('object')
df_final['nombre_pieces_principales'] = df_final['nombre_pieces_principales'].astype('object')
df_final['code_departement'] = df_final['code_departement'].astype('object')
df_final['code_commune'] = df_final['code_commune'].astype('object')
df_final['nombre_lots'] = df_final['nombre_lots'].astype('float64')

In [ ]:
df_final.info()

### Analyse exploratoire pour certains variables importants!

* L'objectif de cette étape est de savoir la distribution des variables importants dans cette base et de voir la relation entre les variables explicatives dans ce jeu avec le prix de l'immobilière (la valeur foncière).
* Nous commençons par l'étude de la distribution et la répartition des variables qualitatives et quantitatives dans ce jeu de données et ensuite nous allons vérifier la rélation (la corrélation) entre les variables à la fin de cette étape!

In [ ]:
df_final['type_local'].value_counts(dropna = False, normalize = True).head()

In [ ]:
#Créer les graphiques!
plt.figure(figsize=(10,10))
labels = ['Donées Manquantes', 'Maison', 'Appartement', 'Dépendance', 'Local industriel. commercial ou assimilé']
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#fffc52']
area = [41, 21.4, 18, 12.9, 0.37]
#area = [91.93, 6.88, 0.82, 0.28,0.056]
explode = (0.05,0.05,0.1,0.1,0.1)
plt.pie(area, labels=labels, explode=explode, colors= colors, startangle=65, autopct='%1.1f%%',shadow='True')
plt.title(label= 'Répartition de la variable type_local')
plt.legend()
plt.show()

* Ici, on voit que pour les transactions immobilières en 2020, les types de biens d'ici sont répartie différement!
                   - 43.8% les transactions n'a pas été saisi par le type de biens.
                   - 22.8% les transantions concerne les maisons en France.
                   - 19.2% les transantions concerne les appartements en France.
                   - Les transactions des bureaux/commerces/entrepôts sont juste 0.4% par rapport la volume totale!

In [ ]:
df_final['code_type_local'].value_counts(dropna = False, normalize = True).head()

In [ ]:
# Répartition de la nature de mutation du bien!
df_final['nature_mutation'].value_counts(dropna = False, normalize = True).head()

In [ ]:
# Visualiser la répartition de la nature de mutation
plt.figure(figsize=(10,10))
labels = ['Vente', 'VEFA', 'Echange', 'Vente Terrain', 'Adjudication']
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#fffc52']
area = [92, 6.8, 0.82, 0.28, 0.56]
#area = [91.93, 6.88, 0.82, 0.28,0.056]
explode = (0.05,0.05,0.2,0.3,0.4)
plt.pie(area, labels=labels, explode=explode, colors= colors, startangle=65, autopct='%1.1f%%',shadow='True')
plt.legend()
plt.show()

Ici, nous trouvons que 91% les tranctions immobilières en 2020 pour tous les régions en France sont les ventes immobilières. La vente en VEFA est juste 6.8% et les restes sont moins de 2% pour les autres types de transactions.
Du coup, nous serons travailler sur la Vente dans ce jeu de données (en excluant tous les autres types de transactions).

In [ ]:
#Traiter les données manquantes

In [ ]:
# L'imputation des données manquantes
df_final['adresse_numero'] = df_full20['adresse_numero'].fillna("None")
df_final['adresse_nom_voie'] = df_full20['adresse_nom_voie'].fillna("None")
df_final['adresse_code_voie'] = df_full20['adresse_code_voie'].fillna("None")
df_final['code_type_local'] = df_full20['code_type_local'].fillna("None")
df_final['type_local'] = df_full20['type_local'].fillna("None")
df_final['valeur_fonciere'] = df_final['valeur_fonciere'].transform(lambda x: x.fillna(x.mean()))
df_final['surface_reelle_bati'] = df_full20['surface_reelle_bati'].fillna(0)
df_final['nombre_pieces_principales'] = df_full20['nombre_pieces_principales'].fillna(0)
df_final['longitude'] = df_full20['longitude'].fillna(0)
df_final['latitude'] = df_full20['latitude'].fillna(0)
df_final['surface_reelle_bati'] = df_full20['surface_reelle_bati'].fillna(0)
df_final['surface_terrain'] = df_final['surface_terrain'].fillna(0)

In [ ]:
#Verifier s'il y a des données manquantes dans la base
df_final.isnull().sum()/len(df_full20)*100

In [ ]:
#3) Filtrer les données pour avoir une base qui contient que les données pour la local commercial, industriel et assimilé
filtered_df = df_final[(df_final["nature_mutation"] == 'Vente')  &  (df_final["code_type_local"] == 4)]

In [ ]:
filtered_df.head(3)

In [ ]:
#Vérifier le nombre des variables quantitatives et qualitatives dans notre jeu de donnée
num_vars = filtered_df.dtypes[filtered_df.dtypes != "object"].index
cat_vars = filtered_df.dtypes[filtered_df.dtypes == "object"].index

print("Nombres des variables quantitatives: ", len(num_vars))
print("Nombres des variables qualitatives: ", len(cat_vars))

In [ ]:
#Créer une fonction pour calculer la corrélation entre les variables quantitatives
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['valeur_fonciere'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
    
features = num_vars
spearman(filtered_df, features)

**** Pour analyser la corrélation entre les variables dans notre jeu de données, nous utlisons la test spearman et pearson pour savoir la rélation entre les variables quantitatives.
**** Ici, nous trouvons que la valeur foncière est corrélé possitivement avec la surface du terrain et la surface reelle du bâtiment. C'est à dire plus la surface est grande, plus la valeur est cher!

In [ ]:
#Matrice de corrélation entre les variables (heatmap)
corrmat = filtered_df.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

**Pour étudier la corrélation entre une variable quantitative et une variable qualitative, nous pouvons utiliser le box-plot pour voir la rélation entre eux. Ici, j'étudie la relation entre le valeur foncière et le code_departement. L'idée est pour voir si la localisation géographique du bien a une impact sur le prix de transaction immobilière. Les commentaires se trouvent en dessous du graph.**

In [ ]:
#Box Plot
data = pd.concat([filtered_df['valeur_fonciere'], filtered_df['code_departement']], axis=1)
f, ax = plt.subplots(figsize=(30, 30))
fig = sns.boxplot(x=filtered_df['code_departement'], y=filtered_df['valeur_fonciere'], data=data)
fig.axis(ymin=0, ymax=65000000);

* Grâce à ce graphique, je vois que la valeur foncière est forte corrélé avec la localisation du bien. Par exemple, pour les déparments dans la régions Ile-de-France, on voit la différent très claire par rapport les autres départements. Les biens dans ces départements a été vendu plus chers par rapport les autres régions de la France. On peut conclure que, pour les prix immobilière en France, la régions Ile-de-France est un cas spécialisé en terme de prix, et la volume de transaction!

* Pour les autres départments en France, on trouve que les département Alpes-maritimes - Nice, Bouches-du-Rhône - Marseille, Haute-garonne - Toulouse, Loire-atlantique - Nantes, Nord - Lille, Gironde - Bordeaux, Rhône - Lyon, les prix sont toujours plus haut que les autres régions de la France (sauf Ile-de-France). 
* Cela est compréhensible, parce que ce sont les plus grands métropoles françaises ou les marché immobiliers sont tendu. Par exemple pour le marché des bureaux, lyon prend la 2 ème position juste après le Grand Paris! 

In [ ]:
import scipy.stats as st
sns.distplot(filtered_df['valeur_fonciere'])

print("Skewness: %f" % filtered_df['valeur_fonciere'].skew())
print("Kurtosis: %f" % filtered_df['valeur_fonciere'].kurt())

On voit ici que la distribution de la valeur fonciere est sysmétrie positivement. Cela indique une distribution décalée à gauche de la médiane, et donc une queue de distribution étalée vers la droite. Dans les études de prix immobilier (notament pour la modélisation et l'estimation prix immobilière), normalement nous transformons le prix en logarithme (log_normale) et de plus, on va s'intéresser au prix par m2 plutôt que le prix total d'une transaction. 

Par ailleurs, on trouves que le type de bien ici sont redondances (il comprend les bureaux, les commerces, les entrepôt) dont cela donne des valeurs foncieres très hétérogènes! 

In [ ]:
#Ici, nous transformons la valeur fonciere en log pour voir la distribution de cette variable: 
import scipy
from scipy.stats import norm
y = np.log(filtered_df['valeur_fonciere'])
sns.distplot(y, fit=norm);
fig = plt.figure()

### 3) Pour la classification les bureaux, commerces et entrepots à partir de la variable type_local je vois que:
 - Avec seulements les données disponibles dans ce jeu de données, pour classfier les types des bâtiments nous pourrons utiliser certaines méthodes comme la K-Nearest Neighboor ou le Clustering mais personnellement, je trouves que les résultats ne seronts pas signigicatives vu que nous n'avons pas beaucoup des informations concernant les caractéristiques du bien pour faire la classification.
 - De l'autre côté, pour chaque bien on a les données géolocalisée (variables coordinates), c'est une information très important. À partir de la variable cordinnates, nous pouvons trouver les images "street view" pour chaque bâtiment avec l'API Google Street View Static. Et puis, nous pouvons collecter tous les images "façade" de chaque bâtiments et créer une dataset contient ques les images.
 - Avec ce dataset, nous pouvons faire ensuite une modèle Réseaux Neuronal Convolutif pour faire la classification des bâtiments à partir de ces images (Il existe aussi d'autres techniques classification comme la SVM, RandomForest Classifier, Regression Logistique Multinomial, etc.).
 - Mais vu que l'API de google n'est pas gratuit pour collecter les images. Donc, j'ai essayé de faire une exemple en collectant une base des images qui contients ques les photos de façades pour les batiments en Etats-Unis. Ce dataset contient 6600 images (en trois classe: industrial, office building, retail) pour les données apprentisage et 845 images pour la validation (3 class: industrial, office building, retail). 